<a href="https://colab.research.google.com/github/Gokul0Krishna/exp/blob/main/Copy_of_exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip "/content/football players.v2i.coco.zip"

Archive:  /content/football players.v2i.coco.zip
   creating: football players.v2i.coco/
  inflating: football players.v2i.coco/README.dataset.txt  
  inflating: football players.v2i.coco/README.roboflow.txt  
   creating: football players.v2i.coco/test/
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0009_jpg.rf.8c94b6f7e2f93d77bf33fa4e740b90e0.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0011_jpg.rf.dd8cce210aef4b83e11dd8e851add934.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0012_jpg.rf.b5165914840af0159721c1b774c0f743.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0024_jpg.rf.6032a83254029bf74e2710a86f9f36cb.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0034_jpg.rf.cdfec2540651bc946b57fc5b9f05dcf9.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0038_jpg.rf.467cd1fbc8e0727dbcf5e0c1b6dcd271.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0051_jpg.rf.50556582ff7415

In [2]:
!git clone https://github.com/facebookresearch/detr.git
%cd detr
!pip install -r requirements.txt
!pip install pycocotools matplotlib

Cloning into 'detr'...
remote: Enumerating objects: 265, done.
remote: Total 265 (delta 0), reused 0 (delta 0), pack-reused 265 (from 1)
Receiving objects: 100% (265/265), 21.19 MiB | 17.87 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/detr
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-install-vji34w7n/pycocotools_5a2a81eded104b4895caf416ff392f69
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-install-vji34w7n/pycocotools_5a2a81eded104b4895caf416ff392f69
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-install-vji34w7n/panopticapi_416e2b2bf4804d029d7a4eccf1494669
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/panopticapi.git /tmp/pip-install-vji34w7n/panopticapi_416e2b2bf4804d029d7a4eccf14946

In [9]:
import torch
from torchvision.datasets import CocoDetection
import torchvision.transforms as T
from pycocotools.coco import COCO
import os
from PIL import Image

class CustomCocoDataset(CocoDetection):
    def __init__(self, root, annFile, transforms=None):
        super().__init__(root, annFile)
        self._transforms = transforms
        self.coco = COCO(annFile)

        # Map cat_id -> 0-based contiguous label
        self.cat_ids = sorted(self.coco.getCatIds())
        self.cat_id_to_label = {cat_id: idx for idx, cat_id in enumerate(self.cat_ids)}
        print("Label Mapping:", self.cat_id_to_label)

    def __getitem__(self, idx):
        img, anns = super().__getitem__(idx)

        boxes, labels = [], []
        for ann in anns:
            cat_id = ann['category_id']
            if cat_id not in self.cat_id_to_label:
                continue
            label = self.cat_id_to_label[cat_id]
            bbox = ann["bbox"]
            bbox[2] += bbox[0]
            bbox[3] += bbox[1]
            boxes.append(bbox)
            labels.append(label)

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}
        if self._transforms:
            img = self._transforms(img)
        return img, target


In [24]:
import torch
from types import SimpleNamespace
import sys
sys.path.append("/content/detr")  # or wherever your `detr/` is located

from models import build_model

# Define args
args = SimpleNamespace(
    dataset_file='custom',
    backbone='resnet50',
    device='cuda',
    dilation=False,
    position_embedding='sine',
    num_classes = 20 if args.dataset_file != 'coco' else 91,
    num_queries=100,
    hidden_dim=256,
    enc_layers=6,
    dec_layers=6,
    dim_feedforward=2048,
    dropout=0.1,
    nheads=8,
    pre_norm=False,
    return_interm_layers=False,
    masks=False,
    lr_backbone=1e-5,
    aux_loss=True,
    set_cost_class=2,
    set_cost_bbox=5,
    set_cost_giou=2,
    bbox_loss_coef=5,
    giou_loss_coef=2,
    mask_loss_coef=1,
    dice_loss_coef=1,
    eos_coef=0.1,
    losses=['labels', 'boxes', 'cardinality']
)

# Build model
model, criterion, postprocessors = build_model(args)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()


DETR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, ou

In [22]:
transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225]),
])

train_dataset = CustomCocoDataset(
    "/content/football players.v2i.coco/train",
    "/content/football players.v2i.coco/_trainannotations.coco.json",
    transforms=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x))
)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
Label Mapping: {0: 0, 1: 1, 2: 2, 3: 3}


In [25]:
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

from models.matcher import HungarianMatcher
from models.detr import SetCriterion

NUM_REAL_CLASSES = 2
NUM_CLASSES = NUM_REAL_CLASSES + 1  # 3 (include no-object class)

# SetCriterion
matcher = HungarianMatcher()
weight_dict = {'loss_ce': 1, 'loss_bbox': 5, 'loss_giou': 2}
losses = ['labels', 'boxes', 'cardinality']
criterion = SetCriterion(
    num_classes=NUM_CLASSES,
    matcher=matcher,
    weight_dict=weight_dict,
    eos_coef=0.1,
    losses=losses
)

criterion.to(device)  # ✅ THIS LINE FIXES THE DEVICE MISMATCH

optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(5):
    model.train()
    total_loss = 0.0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(images)
        print(outputs['pred_logits'].shape)
        loss_dict = criterion(outputs, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


torch.Size([2, 100, 21])


RuntimeError: weight tensor should be defined either for all or no classes

In [ ]:
torch.save(model.state_dict(), "detr_custom.pth")